In [34]:
## Capstone Notebook  for collaboration restricted to non sensitive capstone activities

In [35]:
import numpy as np

In [36]:
# The code was removed by Watson Studio for sharing.

In [37]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [38]:
!pip install bs4

In [39]:

import urllib

toronto_zip_uri = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
try:
    net_file = urllib.request.urlopen(toronto_zip_uri)
    buffer = net_file.read() 
except IOError as ioe:
    print(ioe)

In [40]:
import pandas as pd
import re

def print_df(df):
    with pd.option_context('display.expand_frame_repr',False,'display.max_colwidth',47):
        print(df)

def table_to_zip_df(html_table_tag):
    rows_header = [re.compile('<[tdh]*>\s*[<a]*(["\s\w=/]*[>]*?["\s\w=/]*)[</a>]*\s*</[tdh]*>'),
                   re.compile('<tr>\s*<[tdh]*>[\s\S]*</[tdh]*>\s*</tr>')]
                  
    rows = []
    
    header_horizontal_slice = rows_header[1].findall(str(html_table_tag))[0].split('</tr>')
    
    for index in range(len(header_horizontal_slice)):
                  header_horizontal_slice[index] = header_horizontal_slice[index].replace('<tr>','')
    
                  rows.insert(index,rows_header[0].findall(header_horizontal_slice[index]))
    for row_index in range (len(rows)):
        for col_index in range(len(rows[row_index])):
                               rows[row_index][col_index] = rows[row_index][col_index].replace('\n','')
                               rows[row_index][col_index] = re.sub('[\s\S]*>','',rows[row_index][col_index])

    zip_df = pd.DataFrame(data = rows[1:],columns = rows[0])
    return zip_df
    
    
from bs4 import BeautifulSoup

soup = BeautifulSoup(buffer,'html.parser')    
for table in soup.find_all('table'):
    table_df = table_to_zip_df(table)
    print_df(table_df) 
    break

    Postcode           Borough                                   Neighbourhood
0        M1A      Not assigned                                    Not assigned
1        M2A      Not assigned                                    Not assigned
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
7        M7A      Not assigned                                            None
8        M8A      Not assigned                                    Not assigned
9        M9A  Downtown Toronto                                            None
10       M1B              None                                            None
11       M1B              None                      

In [41]:
table_df = table_df[table_df['Borough'].isna() != True]
print_df(table_df)

    Postcode           Borough                                   Neighbourhood
0        M1A      Not assigned                                    Not assigned
1        M2A      Not assigned                                    Not assigned
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
7        M7A      Not assigned                                            None
8        M8A      Not assigned                                    Not assigned
9        M9A  Downtown Toronto                                            None
12       M2B      Not assigned                                    Not assigned
13       M3B        North York                      

In [42]:
table_df = table_df[table_df['Borough'] != 'Not assigned']
print_df(table_df)

    Postcode           Borough                                   Neighbourhood
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
9        M9A  Downtown Toronto                                            None
13       M3B        North York                                 Don Mills North
14       M4B         East York                                            None
15       M4B         East York                                            None
16       M5B  Downtown Toronto                                         Ryerson
17       M5B  Downtown Toronto                                 Garden District
18       M6B        North York                      

In [43]:
for row in range(table_df.shape[0]):
    if table_df.iloc[row,2] is None: 
        table_df.iloc[row,2] = table_df.iloc[row,1]
    elif table_df.iloc[row,2] == 'Not Assigned':
        table_df.iloc[row,2] = table_df.iloc[row,1]       

print_df(table_df)        

    Postcode           Borough                                   Neighbourhood
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
9        M9A  Downtown Toronto                                Downtown Toronto
13       M3B        North York                                 Don Mills North
14       M4B         East York                                       East York
15       M4B         East York                                       East York
16       M5B  Downtown Toronto                                         Ryerson
17       M5B  Downtown Toronto                                 Garden District
18       M6B        North York                      

In [44]:

table_df = table_df.transform({'Postcode': lambda x : x,'Borough':lambda x : " "+ x +",",'Neighbourhood': lambda x : " " + x + ","})

table_df = table_df.groupby(table_df['Postcode']).sum()

table_df = table_df.reset_index()  
print_df(table_df)


   Postcode                                         Borough                                   Neighbourhood
0       M1E                                      Guildwood,                                      Guildwood,
1       M1H                                      Cedarbrae,                                      Cedarbrae,
2       M1J                            Scarborough Village,                            Scarborough Village,
3       M1K                  East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,
4       M1L                                       Clairlea,                                       Clairlea,
5       M1M           Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,
6       M1N                    Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,
7       M1P           Dorset Park, Scarborough Town Centre,           Dorset Park, Scarborough Town Centre,
8       M1T                 

In [45]:
def rem_dupl_ent_post_sum(x):
        for index in range(x.shape[0]):
            val_str = x.iloc[index]
            val_list = val_str.split(',')
            repl_dict = {}
            repl_str = ''
            for value in val_list:
                repl_dict.update({value:''})
                
            for key_val,val in repl_dict.items():
                if len(repl_str) == 0:
                    repl_str = key_val
                else:
                    repl_str = repl_str + ',' + key_val
            x.iloc[index] = repl_str
        return x

In [46]:
tabl_df = table_df.transform({'Postcode' : lambda x : x,'Borough': rem_dupl_ent_post_sum,'Neighbourhood': rem_dupl_ent_post_sum})
print_df(table_df)

   Postcode                                 Borough                                   Neighbourhood
0       M1E                              Guildwood,                                      Guildwood,
1       M1H                              Cedarbrae,                                      Cedarbrae,
2       M1J                    Scarborough Village,                            Scarborough Village,
3       M1K          East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,
4       M1L                               Clairlea,                                       Clairlea,
5       M1M   Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,
6       M1N            Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,
7       M1P   Dorset Park, Scarborough Town Centre,           Dorset Park, Scarborough Town Centre,
8       M1T               Clarks Corners, Sullivan,                       Clarks Corners, Sullivan,


In [47]:
!pip install pgeocode

In [48]:
import pgeocode


nomi = pgeocode.Nominatim('ca')

df_pc = nomi.query_postal_code(table_df['Postcode'].to_list())
df_pc = df_pc[['postal_code','latitude','longitude']]
df_pc.columns = ['Postcode','latitude','longitude']
df_pc

table_df = pd.merge(table_df,df_pc)
            
table_df = table_df.dropna()
print_df(table_df)

   Postcode                                 Borough                                   Neighbourhood  latitude  longitude
0       M1E                              Guildwood,                                      Guildwood,   43.7678   -79.1866
1       M1H                              Cedarbrae,                                      Cedarbrae,   43.7686   -79.2389
2       M1J                    Scarborough Village,                            Scarborough Village,   43.7464   -79.2323
3       M1K          East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,   43.7298   -79.2639
4       M1L                               Clairlea,                                       Clairlea,   43.7122   -79.2843
5       M1M   Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,   43.7247   -79.2312
6       M1N            Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,   43.6952   -79.2646
7       M1P   Dorset Park, Scarb

In [49]:
print(table_df.shape)

(92, 5)
